In [6]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.5 MB/s eta 0:00:00


STEP 1 — Imports

In [7]:
import pandas as pd
import numpy as np
from faker import Faker
from datetime import datetime, timedelta
import random


STEP 2 — Initialize

In [3]:
fake = Faker()
np.random.seed(42)
random.seed(42)

 STEP 3 — Generate 500 Assets

In [22]:
num_assets = 500

asset_types = ['Laptop', 'Desktop', 'Server', 'Network Device', 'Storage']
departments = ['Finance', 'IT', 'HR', 'Operations', 'Risk', 'Compliance']
statuses = ['Active', 'In Maintenance', 'Retired']
criticality_levels = ['High', 'Medium', 'Low']

assets_data = []

for i in range(1, num_assets + 1):
    purchase_date = fake.date_between(start_date='-8y', end_date='-6m')
    warranty_expiry = purchase_date + timedelta(days=365*3)
    asset_type = random.choice(asset_types)

    if asset_type == 'Laptop':
        cost = random.randint(1000, 2500)
    elif asset_type == 'Desktop':
        cost = random.randint(800, 2000)
    elif asset_type == 'Server':
        cost = random.randint(5000, 15000)
    elif asset_type == 'Network Device':
        cost = random.randint(2000, 8000)
    else:
        cost = random.randint(3000, 12000)

    assets_data.append({
        'asset_id': i,
        'asset_type': asset_type,
        'purchase_date': purchase_date,
        'warranty_expiry': warranty_expiry,
        'cost': cost,
        'department': random.choice(departments),
        'status': random.choice(statuses),
        'vendor': fake.company(),
        'owner': fake.name(),
        'criticality': random.choice(criticality_levels),
        'replacement_priority': random.randint(1,5),
        'business_impact_score': random.randint(1,10)
    })

assets_df = pd.DataFrame(assets_data)

print("Assets shape:", assets_df.shape)

Assets shape: (500, 12)


In [23]:
assets_df = pd.DataFrame(assets_data)
print("✅ Assets Generated:", assets_df.shape)
assets_df.head()

✅ Assets Generated: (500, 12)


,asset_id,asset_type,purchase_date,warranty_expiry,cost,department,status,vendor,owner,criticality,replacement_priority,business_impact_score
0,1,Storage,2023-01-09,2026-01-08,8439,Finance,In Maintenance,Chapman-Arnold,Julie Howell,High,5,8
1,2,Laptop,2024-02-04,2027-02-03,2479,IT,In Maintenance,Munoz-Meadows,Joshua Moore,Low,2,2
2,3,Network Device,2025-05-18,2028-05-17,4171,HR,Retired,Rodriguez-Williams,Natasha Hernandez,Low,4,2
3,4,Server,2022-01-10,2025-01-09,13742,Operations,In Maintenance,Alvarez LLC,Donald Bradford,Low,4,9
4,5,Laptop,2019-03-22,2022-03-21,2264,Finance,Active,Hernandez-Taylor,Shawn Jackson,Low,3,4


STEP 4 — Generate Incidents

In [24]:
incident_data = []
incident_id = 1

for asset_id in assets_df['asset_id']:

    num_incidents = np.random.poisson(2)

    for _ in range(num_incidents):
        incident_data.append({
            'incident_id': incident_id,
            'asset_id': asset_id,
            'incident_date': fake.date_between(start_date='-2y', end_date='today'),
            'severity': random.choices(['Low','Medium','High'], weights=[0.4,0.4,0.2])[0],
            'downtime_hours': random.randint(1,8)
        })
        incident_id += 1

incidents_df = pd.DataFrame(incident_data)
print("✅ Incidents Generated:", incidents_df.shape)
incidents_df.head()

✅ Incidents Generated: (984, 5)


,incident_id,asset_id,incident_date,severity,downtime_hours
0,1,1,2025-09-13,High,1
1,2,1,2024-04-12,High,6
2,3,1,2025-03-04,High,6
3,4,2,2025-12-20,Medium,2
4,5,2,2025-08-28,Low,1


STEP 5 — Generate Software Licenses

In [2]:
software_list = ['MS Office','Adobe','ServiceNow','Power BI','Azure DevOps']

licenses_data = []

for i, software in enumerate(software_list, 1):
    total = random.randint(100,600)
    used = random.randint(50,total)

    licenses_data.append({
        'license_id': i,
        'software_name': software,
        'total_licenses': total,
        'used_licenses': used,
        'expiry_date': fake.date_between(start_date='today', end_date='+2y'),
        'annual_cost': random.randint(20000,150000)
    })

licenses_df = pd.DataFrame(licenses_data)
print("✅ Software Licenses Generated:", licenses_df.shape)
licenses_df.head()

NameError: name 'random' is not defined

 STEP 6 — Save CSV Files

In [26]:
assets_df.to_csv("assets.csv", index=False)
incidents_df.to_csv("incidents.csv", index=False)
licenses_df.to_csv("software_licenses.csv", index=False)

print("✅ All CSV Files Created: assets.csv, incidents.csv, software_licenses.csv")

✅ All CSV Files Created: assets.csv, incidents.csv, software_licenses.csv


In [9]:
!pip install mysql-connector-python pandas



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 55.0 MB/s eta 0:00:00


In [10]:
import mysql.connector
import pandas as pd
from datetime import datetime

Replace these with your MySQL credentials

In [11]:
assets_df = pd.read_csv("assets.csv")
incidents_df = pd.read_csv("incidents.csv")
licenses_df = pd.read_csv("software_licenses.csv")

print("Assets:", assets_df.shape)
print("Incidents:", incidents_df.shape)
print("Licenses:", licenses_df.shape)

Assets: (500, 12)
Incidents: (984, 5)
Licenses: (5, 6)


In [14]:
assets_df['purchase_date'] = pd.to_datetime(assets_df['purchase_date'])
assets_df['asset_age'] = (pd.to_datetime("today") - assets_df['purchase_date']).dt.days / 365

In [15]:

incident_count = incidents_df.groupby('asset_id').size().reset_index(name='incident_count')
assets_df = assets_df.merge(incident_count, how='left', on='asset_id')
assets_df['incident_count'] = assets_df['incident_count'].fillna(0)


In [16]:
print(assets_df.columns)


Index(['asset_id', 'asset_type', 'purchase_date', 'warranty_expiry', 'cost',
       'department', 'status', 'vendor', 'owner', 'criticality',
       'replacement_priority', 'business_impact_score', 'asset_age',
       'incident_count'],
      dtype='object')


In [17]:
downtime = incidents_df.groupby('asset_id')['downtime_hours'].sum().reset_index()
assets_df = assets_df.merge(downtime, how='left', on='asset_id')
assets_df['downtime_hours'] = assets_df['downtime_hours'].fillna(0)


In [18]:
assets_df['age_score'] = assets_df['asset_age'] / assets_df['asset_age'].max()
assets_df['incident_score'] = assets_df['incident_count'] / assets_df['incident_count'].max()
assets_df['downtime_score'] = assets_df['downtime_hours'] / assets_df['downtime_hours'].max()
assets_df['priority_score'] = assets_df['replacement_priority'] / assets_df['replacement_priority'].max()
assets_df['impact_score'] = assets_df['business_impact_score'] / assets_df['business_impact_score'].max()


In [19]:
assets_df['risk_score'] = (
    0.2*assets_df['age_score'] +
    0.3*assets_df['incident_score'] +
    0.2*assets_df['downtime_score'] +
    0.15*assets_df['priority_score'] +
    0.15*assets_df['impact_score']
)

In [20]:
def risk_category(score):
    if score >= 0.7:
        return "High Risk"
    elif score >= 0.4:
        return "Medium Risk"
    else:
        return "Low Risk"

assets_df['risk_category'] = assets_df['risk_score'].apply(risk_category)


In [21]:
threshold = assets_df['risk_score'].quantile(0.8)  # top 20%
assets_df['target_high_risk'] = (assets_df['risk_score'] > threshold).astype(int)

print(assets_df['target_high_risk'].value_counts())


target_high_risk
0    400
1    100
Name: count, dtype: int64


In [22]:
features = [
    'asset_age',
    'incident_count',
    'downtime_hours',
    'business_impact_score',
    'replacement_priority'
]

X = assets_df[features]
y = assets_df['target_high_risk']

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [24]:

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, predictions))
print(classification_report(y_test, predictions))

Accuracy: 0.9333333333333333
              precision    recall  f1-score   support

           0       0.94      0.98      0.96       122
           1       0.91      0.71      0.80        28

    accuracy                           0.93       150
   macro avg       0.92      0.85      0.88       150
weighted avg       0.93      0.93      0.93       150



In [25]:
assets_df['target_high_risk'].value_counts()


,count
target_high_risk,
0,400
1,100


In [26]:
assets_df['predicted_risk_probability'] = model.predict_proba(X)[:,1]

In [27]:
assets_df.to_csv("assets_ai_final.csv", index=False)


In [31]:
print(assets_df[['asset_id','asset_type','department','risk_score','risk_category']].head())


   asset_id      asset_type  department  risk_score risk_category
0         1         Storage     Finance    0.525118   Medium Risk
1         2          Laptop          IT    0.283356      Low Risk
2         3  Network Device          HR    0.221254      Low Risk
3         4          Server  Operations    0.497533   Medium Risk
4         5          Laptop     Finance    0.370190      Low Risk


In [34]:
assets_df.to_csv("assets_ai_final.csv", index=False)
print("Final AI dataset exported!")


Final AI dataset exported!


In [28]:
importance = pd.Series(model.feature_importances_, index=features)
importance_df = importance.reset_index()
importance_df.columns = ['feature', 'importance']
importance_df.to_csv("feature_importance.csv", index=False)


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
